In [81]:
#import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()

In [82]:
#import data
xls_file = pd.ExcelFile('C:\\Users\\sonja tilly.ABERDEEN\\credit spread drivers 0717.xlsx')

**Prep data**

In [83]:
#parse sheet
mthly = xls_file.parse('monthly_transformed')
mthly = mthly.fillna(0.0000001)
mthly.tail(1)

,FDTR Index,VIX Index,USTW$ Index,USGG10YR Index,USGG30YR Index,USGG5YR Index,USYC2Y10 Index,USYC2Y30 Index,USYC5Y10 Index,USYC2Y5Y Index,...,CICRTOT Index,ARDIMOYY Index,M1% YOY Index,SCGRRAI Index,EMB US Equity,SPE AUTO Index,SPE CARD Index,BFCIUS Index,INJCJMOM Index,LEI CHNG Index
NaT,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,0.0,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07


In [84]:
#parse sheet
qly = xls_file.parse('qly1_transformed')
qly = qly.fillna(0.0000001)
qly.head(2)

,GDP CYOY Index
1997-03-31,1.000000e-07
1997-06-30,-6.976744e-02


In [85]:
#parse sheet
qly2 = xls_file.parse('qly2_transformed')
qly2 = qly2.fillna(0.0000001)
qly2.head(1)

,SLDETIGT Index,SLDETGTS Index
1997-01-31,1.000000e-07,1.000000e-07


In [106]:
#parse sheet
dly = xls_file.parse('dly_transformed')
dly = dly.fillna(0.0000001)
dly.head(1)

,GFSIFFND Index,XLF US Equity,XLE US Equity,XLK US Equity,XLV US Equity,XLI US Equity,XLY US Equity,XLB US Equity,CDX HY CDSI GEN 5Y SPRD Corp
1998-12-31,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07


In [107]:
upsampled = qly.resample('D').mean().fillna(method='bfill')
upsampled.head()

,GDP CYOY Index
1997-03-31,1.000000e-07
1997-04-01,-6.976744e-02
1997-04-02,-6.976744e-02
1997-04-03,-6.976744e-02
1997-04-04,-6.976744e-02


In [108]:
upsampled2 = qly2.resample('D').mean().fillna(method='bfill')
upsampled2.head()

,SLDETIGT Index,SLDETGTS Index
1997-01-31,1.000000e-07,1.000000e-07
1997-02-01,2.142857e-01,-5.142857e-01
1997-02-02,2.142857e-01,-5.142857e-01
1997-02-03,2.142857e-01,-5.142857e-01
1997-02-04,2.142857e-01,-5.142857e-01


In [89]:
#upsampled3 = dly.resample('M').mean().fillna(method='bfill')
#upsampled3.head()

In [110]:
#merge dataframes

merged = pd.concat([mthly, upsampled, upsampled2, dly], axis=1)
df = merged.resample('M').mean().fillna(method='bfill')

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 247 entries, 1997-01-31 to 2017-07-31
Data columns (total 54 columns):
FDTR Index                      247 non-null float64
VIX Index                       247 non-null float64
USTW$ Index                     247 non-null float64
USGG10YR Index                  247 non-null float64
USGG30YR Index                  247 non-null float64
USGG5YR Index                   247 non-null float64
USYC2Y10 Index                  247 non-null float64
USYC2Y30 Index                  247 non-null float64
USYC5Y10 Index                  247 non-null float64
USYC2Y5Y Index                  247 non-null float64
CPI YOY Index                   247 non-null float64
FDIDFDMO Index                  247 non-null float64
USURTOT Index                   247 non-null float64
NFP TCH Index                   247 non-null float64
USEMNCHG Index                  247 non-null float64
IP CHNG Index                   247 non-null float64
MTIBCHNG Index              

C:\Users\sonja tilly.ABERDEEN\AppData\Local\Continuum\Anaconda2\envs\py35\lib\site-packages\pandas\indexes\base.py:1877: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


**Create new features**

In [111]:
#calculate z scores

cols = list(df.columns)

for col in cols:
    col_zscore = col + '_zscore'
    df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [112]:
#calculate rolling 3M z-score

cols = list(df.columns)

for col in cols:
    col_zscore = col + '_zscore'
    mean = df[col].rolling(center=False, window=3).mean()
    std = df[col].rolling(center=False, window=3).std()
    df[col_zscore] = (df[col] - mean)/std



In [113]:
df.columns

Index(['FDTR Index', 'VIX Index', 'USTW$ Index', 'USGG10YR Index',
       'USGG30YR Index', 'USGG5YR Index', 'USYC2Y10 Index', 'USYC2Y30 Index',
       'USYC5Y10 Index', 'USYC2Y5Y Index',
       ...
       'SLDETGTS Index_zscore_zscore', 'GFSIFFND Index_zscore_zscore',
       'XLF US Equity_zscore_zscore', 'XLE US Equity_zscore_zscore',
       'XLK US Equity_zscore_zscore', 'XLV US Equity_zscore_zscore',
       'XLI US Equity_zscore_zscore', 'XLY US Equity_zscore_zscore',
       'XLB US Equity_zscore_zscore',
       'CDX HY CDSI GEN 5Y SPRD Corp_zscore_zscore'],
      dtype='object', length=162)

In [120]:
#address data lags

#df[['CPI YOY Index','FDIDFDMO Index','USURTOT Index','NFP TCH Index', 'USEMNCHG Index',\
 #  'IP CHNG Index', 'MTIBCHNG Index', 'MGT2TB Index', 'OUTFGAF Index', 'USHBMIDX Index',\
  # 'CONCCONF Index','RSTAMOM Index', 'USTBTOT Index', 'CONSSENT Index', 'CICRTOT Index',\
   #'SPE AUTO Index', 'SPE CARD Index', 'GDP CYOY Index']].shift(3).fillna(0.0000001)

df['CPI YOY Index'] = df['CPI YOY Index'].shift(3)
df['FDIDFDMO Index'] = df['FDIDFDMO Index'].shift(3)
df['NFP TCH Index'] = df['NFP TCH Index'].shift(3)
df['USEMNCHG Index'] = df['USEMNCHG Index'].shift(3)
df['IP CHNG Index'] = df['IP CHNG Index'].shift(3)
df['MTIBCHNG Index'] = df['MTIBCHNG Index'].shift(3)
df['MGT2TB Index'] = df['MGT2TB Index'].shift(3)
df['OUTFGAF Index'] = df['OUTFGAF Index'].shift(3)
df['USHBMIDX Index'] = df['USHBMIDX Index'].shift(3)
df['CONCCONF Index'] = df['CONCCONF Index'].shift(3)
df['RSTAMOM Index'] = df['RSTAMOM Index'].shift(3)
df['USTBTOT Index'] = df['USTBTOT Index'].shift(3)
df['CONSSENT Index'] = df['CONSSENT Index'].shift(3)
df['CICRTOT Index'] = df['CICRTOT Index'].shift(3)
df['SPE AUTO Index'] = df['SPE AUTO Index'].shift(3)
df['GDP CYOY Index'] = df['GDP CYOY Index'].shift(3)

df.head()

,FDTR Index,VIX Index,USTW$ Index,USGG10YR Index,USGG30YR Index,USGG5YR Index,USYC2Y10 Index,USYC2Y30 Index,USYC5Y10 Index,USYC2Y5Y Index,...,SLDETGTS Index_zscore_zscore,GFSIFFND Index_zscore_zscore,XLF US Equity_zscore_zscore,XLE US Equity_zscore_zscore,XLK US Equity_zscore_zscore,XLV US Equity_zscore_zscore,XLI US Equity_zscore_zscore,XLY US Equity_zscore_zscore,XLB US Equity_zscore_zscore,CDX HY CDSI GEN 5Y SPRD Corp_zscore_zscore
1997-01-31,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-02-28,0.000000e+00,7.725118e-02,2.875537e-02,8.852259e-03,2.057916e-03,2.207609e-02,-3.540984e-01,-4.049296e-01,-9.015873e-01,-6.666667e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-03-31,2.500000e-01,4.697380e-02,8.466826e-03,5.084746e-02,4.129087e-02,5.405806e-02,1.011788e-01,-2.305476e-02,1.463415e-01,7.550077e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-04-30,0.000000e+00,-1.036889e-01,1.181038e-02,-2.753796e-02,-1.997988e-02,-2.723262e-02,-1.041215e-01,4.670330e-02,-1.373626e-02,-1.630824e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-31,0.000000e+00,-4.533611e-02,-1.738481e-02,-8.860189e-03,-7.384883e-03,-1.123077e-02,-2.104097e-02,5.270007e-02,-2.013201e-01,7.000000e-02,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#parse sheet
tgt = xls_file.parse('tgt')
y = np.array(tgt).ravel()

In [ ]:
# For each variable in X, calculate VIF and save in dataframe

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif["features"] = df.columns
vif = (vif[vif.vif >= 5])
vif

In [ ]:
#drop features with infinite or above 5 variance inflation factor

X.drop(X[['USGG10YR% Index_zscore', 'USGG30YR% Index_zscore', 'USGG5YR% Index_zscore',\
                  'USTBTOT% Index_zscore', 'GDPCTOT% Index_zscore',\
                  'GPDITOC% Index_zscore', 'GDP CQOQ Index_zscore']], axis=1, inplace=True)

In [ ]:
#split dataset

X_train = X[:196]
y_train = y[:196]
X_test = X[196:]
y_test = y[196:]
X_train.head()

In [ ]:
#try Lasso regression

from sklearn import linear_model

lasso = linear_model.Lasso(alpha=0.21, fit_intercept=False, normalize=False, precompute=False,\
                           copy_X=True, max_iter=1000, tol=0.01, warm_start=False,\
                           positive=False, random_state=42, selection='cyclic')
lasso.fit(X_train, y_train)
lasso.score(X_test,y_test)

In [ ]:
#try RandomForests

from sklearn.ensemble import RandomForestRegressor

tree = RandomForestRegressor(n_estimators=30, criterion='mse',
                            max_depth=4, min_samples_split=2, 
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.30,
                            max_features='auto',
                            max_leaf_nodes=None,
                            min_impurity_split=1e-07,
                            bootstrap=True, oob_score=False, n_jobs=-1,
                            random_state=None, verbose=0,
                            warm_start=False)
tree.fit(X_train, y_train)
tree.score(X_test, y_test)